In [274]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TokenClassificationPipeline
from transformers import pipeline
import pandas as pd


In [275]:
# 1. Cargar los datos
df = pd.read_excel("DatosTecnicaTurboshop.xlsx", sheet_name="F3")
df.head(20)

,Codigo,Nombre,Aplicaciones
0,2996724-5,RADIADOR MT L300 2.4 92-98+2.0-2.5 96-11 T/M C...,NaN
1,041010021,ACEITE 75W90 OPTIMUS DIF+CAJA T/M 1LT SEMISIN 648,NaN
2,OK7AA-52-910,PTA FRONTAL LH 2005> GENUINO,NaN
3,280085,PRENSA EMB HY SON EF VALEO,NaN
4,71911-43900,PTA FRONTAL 98-2004 LH VAN,NaN
5,21443-33004,RET CIG TRAS HY H-100 80*96*9 KOREA,NaN
6,24535532,MANILLA N300 ASIDERO,NaN
7,213109151,RADIADOR SS ACTYON 2.0 07-11 T/M GEN,NaN
8,0016758D,FOTO TRAS KIA CARENS 07-11 (C/DETALLE) CHINA,NaN
9,54501-FD000,BANDEJA KIA RIO 03-05 RH KOREA,NaN


In [276]:
# Cargar tu modelo entrenado (ajusta los nombres de ruta según corresponda)
model_path = "train/modelo_ner_repuestos"  # carpeta con config, tokenizer, pytorch_model.bin, etc.

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy='max')


Device set to use mps:0


In [277]:
def expandir_años(entidades):
    """
    Expande rangos de años SOLO si coinciden con los formatos específicos.
    Si no hay coincidencias válidas, retorna una lista vacía [].
    
    Formatos aceptados:
    - 2015 a 2020, 2015-2020, 2015-20, 15-2020, 15-20
    - 2020 (año individual de 4 dígitos)
    - 90, 05 (año individual de 2 dígitos, entre 00-99)
    
    Args:
        entidades: Diccionario con entidades reconocidas (debe contener clave "año")
    
    Returns:
        Lista de años expandidos (únicos y ordenados) o [] si no hay formatos válidos.
    """
    años_expandidos = []
    
    for año in entidades.get("año", []):
        año = año.strip()
        
        # Verificar si el formato es válido antes de procesarlo
        es_formato_valido = False
        año_normalizado = None
        
        # Caso 1: Rangos con separadores (a, l, /, |, \, -)
        if any(sep in año for sep in [' a ', ' l ', '/', '|', '\\', '-']):
            año_normalizado = año.replace(' a ', '-').replace(' l ', '-').replace('/', '-').replace('|', '-').replace('\\', '-')
            partes = año_normalizado.split('-')
            
            # Caso 2015-2020 o 2015 a 2020
            if len(partes) == 2 and len(partes[0]) == 4 and len(partes[1]) == 4:
                es_formato_valido = True
            
            # Caso 2015-20
            elif len(partes) == 2 and len(partes[0]) == 4 and len(partes[1]) == 2:
                es_formato_valido = True
            
            # Caso 15-2020
            elif len(partes) == 2 and len(partes[0]) == 2 and len(partes[1]) == 4:
                es_formato_valido = True
            
            # Caso 15-20
            elif len(partes) == 2 and len(partes[0]) == 2 and len(partes[1]) == 2:
                es_formato_valido = True
        
        # Caso 2: Año individual de 4 dígitos (2020)
        elif año.isdigit() and len(año) == 4:
            es_formato_valido = True
            año_normalizado = año
        
        # Caso 3: Año individual de 2 dígitos (90, 05)
        elif año.isdigit() and len(año) == 2:
            es_formato_valido = True
            año_normalizado = año
        
        # Si el formato NO es válido, ignorar este año
        if not es_formato_valido:
            continue
        
        # Procesar solo si el formato es válido
        if '-' in año_normalizado:
            inicio, fin = año_normalizado.split('-')
            
            # Convertir años de 2 dígitos a 4 (15 → 2015, 90 → 1990)
            if len(inicio) == 2:
                siglo_inicio = '19' if int(inicio) >= 90 else '20'
                inicio = siglo_inicio + inicio
            if len(fin) == 2:
                siglo_fin = '19' if int(fin) >= 90 else '20'
                fin = siglo_fin + fin
            
            try:
                años_expandidos.extend([str(y) for y in range(int(inicio), int(fin)+1)])
            except ValueError:
                continue
        else:
            # Manejar año individual (2 o 4 dígitos)
            if len(año_normalizado) == 2:
                siglo = '19' if int(año_normalizado) >= 90 else '20'
                año_normalizado = siglo + año_normalizado
            
            if 1900 <= int(año_normalizado) <= 2099:  # Validación adicional
                años_expandidos.append(año_normalizado)
    
    # Eliminar duplicados y ordenar (si hay resultados)
    if años_expandidos:
        return sorted(list(set(años_expandidos)))
    else:
        return []  # Si no hubo formatos válidos

In [278]:
def reconstruir_entidades(ner_output):
    entidades = {"marca": [], "modelo": [], "año": [], "cilindrada": []}
    
    for ent in ner_output:
        label = ent["entity_group"].lower()
        text = ent["word"]

        # Reconstruir cilindrada como "2.0", no "2" "." "0"
        if label == "cilindrada":
            entidades[label].append(text.replace(" ", ""))  # ya vienen unidos si usas aggregation_strategy
        else:
            entidades[label].append(text.upper())
    
    return entidades


In [279]:
# mapping: abreviacion → nombre completo
mapping_marcas = {'HY': 'HYUNDAI', 'SS': 'SSANGYONG', 'CHV': 'CHEVROLET'}
mapping_modelos = {'SON': 'SONATA', 'H1': 'H-1', 'ACTYON': 'ACTYON'}  # debes construir con tus datos


In [467]:
import re

PAISES = {"CHINA", "KOREA", "TAIWAN", "JAPON", "ITALIA", "TAILANDIA", "CH", "KR"}
TRASH = {"PISTON", "REPUESTO", "LT", "SUP", "PRENSA", "EMB", "EF","VALEO", "EMBRAGUE", "JG", "FE", "LH", "SOP", "CROMADO", "PAMAX", "NEGRO", "2WD", "4WD", "NET", "100", "REX",
         "CENTRIFUGO","TW", "DIF", "MANUAL","1LT", "BARRA", "2X", "PTA", "POLEA", "CIGUEÑAL", "4G15", "FOCO", "INYECTORA", "SBK", "RH", "MOLDURA", "FUNDA", "RET", "CIG", "HIDRA",
         "RUEDA", "TECLE", "FOTO", "DIESEL", "AL", "KYB", "HATCH", "BACK", "HID", "TAPA", "DIR", "INF", "ROD", "HIDRA", "CROMADA", "MOTORS", "MAP", "ELECT", "INT"}

def procesar_texto(texto):
    tokens = texto.split()
    tokens_limpios = []
    for tok in tokens:
        tok = tok.strip().upper()
        if len(tok) <= 1:
            continue
        if tok in PAISES or tok in TRASH:
            continue
        if re.fullmatch(r'\W+', tok):
            continue
        tokens_limpios.append(tok)
    return " ".join(tokens_limpios)


In [471]:
def aplicar_ner_a_dataframe(df):
    resultados = []

    # Diccionario de abreviaciones de marcas
    ABREVIACIONES_MARCAS = {
        "MT": "MITSUBISHI",
        "HY": "HYUNDAI",
        "SS": "SSANGYONG",
        "KIA": "KIA",
        "MD": "HYUNDAI"
    }

    # Marcas válidas (abreviaciones + nombres completos)
    MARCAS_VALIDAS = set(ABREVIACIONES_MARCAS.keys()).union(set(ABREVIACIONES_MARCAS.values()))

    for _, fila in df.iterrows():
        texto = fila["Nombre"]
        texto_limpio = procesar_texto(texto)
        ner_output = nlp(texto_limpio)
        entidades = reconstruir_entidades(ner_output)
        años_expandido = expandir_años(entidades)

        if not años_expandido:
            años_expandido = [None]

        # Función para filtrar y normalizar marcas/modelos
        def filtrar_marca_modelo(lista_palabras, es_marca=False):
            if not lista_palabras:
                return None
            
            # Unir palabras y convertir a mayúsculas
            texto = " ".join(lista_palabras).strip().upper()
            
            # Si es marca:
            if es_marca:
                # Aplicar diccionario de abreviaciones
                texto = ABREVIACIONES_MARCAS.get(texto, texto)
                # Si la marca no está en las válidas, descartar
                if texto not in MARCAS_VALIDAS:
                    return None
                # Si contiene símbolos raros (como paréntesis), descartar
                if any(caracter in texto for caracter in {"(", ")", ")", "-", "="}):
                    return None
            
            # Si es modelo, verificar que sea UNA SOLA PALABRA
            elif len(texto.split()) != 1:
                return None
            
            return texto if texto else None

        # Obtener marca y modelo filtrados
        marca_filtrada = filtrar_marca_modelo(entidades.get("marca", []), es_marca=True)
        modelo_filtrado = filtrar_marca_modelo(entidades.get("modelo", []))

        # Solo añadir al resultado si la marca y modelo son válidos
        if marca_filtrada and modelo_filtrado:
            for año in años_expandido:
                resultados.append({
                    "Marca": marca_filtrada,
                    "Modelo": modelo_filtrado,
                    "Cilindrada": "".join(entidades.get("cilindrada", [None]))[:3],
                    "Año": año
                })

    return pd.DataFrame(resultados)

In [472]:
df_resultado = aplicar_ner_a_dataframe(df)


In [473]:
df_resultado.dropna(subset=["Marca"], inplace=True)
df_resultado.dropna(subset=["Modelo"], inplace=True)

df_resultado.reset_index(drop=True, inplace=True)
df_resultado

,Marca,Modelo,Cilindrada,Año
0,HYUNDAI,SON,,None
1,KIA,RIO,,2003
2,KIA,RIO,,2005
3,HYUNDAI,I10,,2015
4,HYUNDAI,I10,,2018
5,KIA,CARENS,,2007
6,KIA,CARENS,,2013
7,SSANGYONG,ACTYON,,2007
8,SSANGYONG,ACTYON,,2011
9,SSANGYONG,ACTYON,,2007


In [474]:
df_resultado.rename(columns={
    'Marca': 'MARCA',
    'Modelo': 'MODELO',
    "Cilindrada": "CILINDRADA",
    "Año": "AÑO"
}, inplace=True)


In [475]:
df_resultado.to_csv("data_limpia/datos_limpios_F3.csv", index=False, encoding='utf-8-sig')